<h1 style="text-align: center; color: RED">Feature Engineering & Data Preprocessing Uygulama</h1>

* Bu kısıma titanic veri setini uçtan uca ele alacaz.Bu veri setinde hedef insanların hayatta kalıp kalmama durmunu modellemektir. Bu kısımda modelleme öncesi gerekli tüm adımları değerlendiricez.

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno #eksik değerler için kullanılan bir kütüphane
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

#Görsel ayaralama için
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [6]:
def load():
    data = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\6.Feature Engineering\\datasets\\titanic.csv")
    return data

In [8]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [10]:
df.shape

(891, 12)

In [13]:
#Değişken isimlerini tek bir formata getirme
df.columns = [col.upper() for col in df.columns]

In [15]:
df.head()

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


<span style="color: blue; font-weight: bold;">1. ADIM: FEATURE ENGİNEERİNG (DEĞİŞKEN MÜHENDİSLİĞİ)</span>

In [19]:
#1-CABİN BOOL
#"Kabini olmayan yolcuların hayatta kalma oranı düşük mü?" gibi analizler yapılabilmek için yeni bir değişken türettik.

df["NEW_CABIN_BOOL"] = df["CABIN"].notnull().astype('int')
#"CABIN" sütunu, yolcuların kabin numarasını içeren bir sütun olabilir. Ancak, bazı yolcuların kabin bilgisi eksik (NaN) olabilir.
#Pandas'ta .notnull() metodu, bir sütundaki eksik olmayan (NaN olmayan) değerleri True olarak işaretler.
#NaN (boş) olan değerler False döndürür.
#.astype('int') Boolean (True/False) değerlerini int (0 veya 1) formatına çevirir.
#Böylece, "CABIN" sütununda boş olmayan (NaN olmayan) değerler 1, NaN olan değerler 0 olarak saklanır.

In [21]:
#2- NAME COUNT
#Bazı veri setlerinde isim uzunluğu ile sosyal statü arasında bir ilişki olabilir.
#Örneğin, daha uzun isimler akademik unvanlar içerebilir (Prof., Dr., Miss. gibi).
#Titanic veri setinde, uzun isimlerin daha çok soylulara ait olduğu ve hayatta kalma oranlarının farklı olduğu görülmüştür.

df["NEW_NAME_COUNT"] = df["NAME"].str.len()
#.str: Pandas’ta bir string (metin) işlemleri yapabilmek için kullanılan bir özelliktir.
#.len(): String'in uzunluğunu (karakter sayısını) döndürür.
#Her bir hücre için .len() çalıştırılır ve isimdeki toplam karakter sayısı hesaplanır.

In [25]:
# 3-NAME WORD COUNT
#Daha fazla kelime içeren isimler unvan (Dr., Prof., Mr., Miss gibi) içerebilir.
#Unvanlar, bir kişinin sosyal statüsü veya mesleği hakkında bilgi verebilir. (Örneğin, Titanic veri setinde unvanlar hayatta kalma oranıyla ilişkilendirildi.)
#ünvan taşıyan yolcuların hayatta kalma oranını tahmin etmek için kullanılabilir.

df["NEW_NAME_WORD_COUNT"] = df["NAME"].apply(lambda x: len(str(x).split(" ")))
#.apply(): Pandas’ta, her bir satırdaki değeri bir fonksiyon yardımıyla işlemek için kullanılır.
#str(x) → Değeri String’e Çevirme. Eğer "NAME" sütununda NaN (eksik) değerler varsa, str() kullanarak bunları metne çeviriyoruz.
#Örneğin, "Alice Wonderland" zaten string olduğu için değişmez ama NaN varsa "nan" olarak dönüşebilir.
#.split(" "), metni boşluklardan ayırarak kelimelere böler."John Doe" → ["John", "Doe"]
#len(...) → Liste Uzunluğunu Hesaplama


In [27]:
# 4-NAME DR
#"Dr" unvanı taşıyan kişilerin davranışları veya tercihleri farklı olabilir.

df["NEW_NAME_DR"] = df["NAME"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))
#[x for x in x.split() if x.startswith("Dr")]:
#List comprehension (liste üreteci) kullanılarak "Dr" ile başlayan kelimeler seçilir.

In [31]:
#5-NAME TİTLE
#Unvanlar kişinin cinsiyeti, mesleği veya sosyal statüsü hakkında bilgi verebilir. Bu da veri analizi ve makine öğrenmesi modelleri için önemli olabilir.

df['NEW_TITLE'] = df.NAME.str.extract(r' ([A-Za-z]+)\.', expand=False)
#str.extract(' ([A-Za-z]+)\.') → Regex ile Unvanları Bulma
#.str.extract(): Pandas’ın düzenli ifadeler (regex) ile metinden belirli bölümleri çıkarmaya yarayan fonksiyonudur.
#' ([A-Za-z]+)\.': Bu düzenli ifade (regex) ile unvanlar bulunur.
#Kod çalıştırıldığında, "NAME" sütunundaki "Dr" ile başlayan kelime sayısı yeni bir sütuna eklenir.
#Ancak expand=False ile çıkarılan değerler direkt olarak tek bir sütun halinde atanır.
#Eğer expand=True kullanılsaydı, sonuç DataFrame formatında dönerdi.

In [33]:
# 6-FAMILY SİZE
#Titanicte aile bireylerine göre hayatta kalma oranları yorumlanabilir.
#Orta büyüklükteki aileler (2-4 kişi) hayatta kalma şansı daha yüksek olabilir.
#Tek başına seyahat edenler (1 kişi) daha düşük hayatta kalma oranına sahip olabilir.
#Büyük aileler (5+ kişi) daha düşük hayatta kalma oranına sahip olabilir.

df["NEW_FAMILY_SIZE"] = df["SIBSP"] + df["PARCH"] + 1
#SIBSP  Kişinin yanında seyahat eden kardeşleri (sibling) ve eşi (spouse) sayısı
#PARCH Kişinin yanında seyahat eden ebeveynleri (parent) ve çocukları (children) sayısı
#+1 de kişinin kendisi
#Bu kod, bir kişinin seyahat ettiği aile büyüklüğünü bulmak için aşağıdaki işlemi yapar

In [35]:
# 7-AGE-PCLASS
# yaş ile bilet sınıfını bir araya getirerek yaşın bilet sınıfı üzerindeki etkisini hesaplamaya çalışır.
#1 Sınıfsal Farklılıkları Daha Belirgin Hale Getirmek
#3. sınıf yolcular daha genç olabilir mi?
#1. sınıf yolcular genellikle daha yaşlı mı?
#Hayatta Kalma Oranı Üzerinde Etkisi Olabilir
#Sınıfsal farklılıkları ve yaşın etkisini daha iyi analiz etmeye yardımcı olur.
df["NEW_AGE_PCLASS"] = df["AGE"] * df["PCLASS"]

In [37]:
# 8-İS ALONE
#Kişi gemide tekmi yoksa yanında bir akarabası var mı?
#Akrabasının olması hayatta kalmasını etkilemişmidir?
df.loc[((df['SIBSP'] + df['PARCH']) > 0), "NEW_IS_ALONE"] = "NO"
df.loc[((df['SIBSP'] + df['PARCH']) == 0), "NEW_IS_ALONE"] = "YES"

In [39]:
# 9-AGE LEVEL
#Genç yolcular farklı davranışlar sergileyebilir (daha az hayatta kalma şansı veya farklı seyahat tercihleri olabilir).
#Olgun yolcular ve yaşlı yolcular da farklı özellikler gösterebilir (belki daha fazla hayatta kalma şansı).
df.loc[(df['AGE'] < 18), 'NEW_AGE_CAT'] = 'young'
df.loc[(df['AGE'] >= 18) & (df['AGE'] < 56), 'NEW_AGE_CAT'] = 'mature'
df.loc[(df['AGE'] >= 56), 'NEW_AGE_CAT'] = 'senior'
#Yeni oluşturulan NEW_AGE_CAT sütunu, young, mature, ve senior gibi yaş gruplarını temsil eder.

In [41]:
# 10-SEX X AGE
#yaş (Age) ve cinsiyet (Sex) bilgilerini kullanarak yolcuları daha spesifik kategorilere ayırmak için yazılmıştır.
# Yeni oluşturulan NEW_SEX_CAT sütunu, cinsiyet ve yaş kombinasyonuna göre aşağıdaki gibi 6 farklı kategoriyi içerir
#Veri analizi ve makine öğrenmesi için yaş ve cinsiyet bilgilerini daha anlamlı gruplara ayırarak yeni bir özellik (feature) yaratmak,
#bu değişkenlerin etkisini daha iyi anlayabilmemizi sağlar.
#Cinsiyet grupları: Kadınlar ve erkekler arasında, yaşa bağlı olarak farklı yaşam tarzları, seyahat tercihleri veya hayatta kalma oranları olabilir.
#Yaş grupları: Yaşın farklı gruplarda nasıl farklı sonuçlar doğurduğu incelenebilir. Örneğin, genç erkeklerin, mature kadınların ve yaşlı bireylerin hayatta kalma oranları farklı olabilir.
df.loc[(df['SEX'] == 'male') & (df['AGE'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['SEX'] == 'male') & (df['AGE'] > 21) & (df['AGE'] < 50), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['SEX'] == 'male') & (df['AGE'] >= 50), 'NEW_SEX_CAT'] = 'seniormale'
df.loc[(df['SEX'] == 'female') & (df['AGE'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['SEX'] == 'female') & (df['AGE'] > 21) & (df['AGE'] < 50), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['SEX'] == 'female') & (df['AGE'] >= 50), 'NEW_SEX_CAT'] = 'seniorfemale'

In [45]:
df.shape

(891, 22)

In [47]:
def grab_col_names(dataframe, cat_th=10, car_th=20):#parametrelerin ön tanımlı değerlerini burda belirttik

    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['object', 'category', 'bool']]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].dtypes in ['int64', 'float64'] and dataframe[col].nunique() < cat_th]
    cat_but_car = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['object', 'category'] and dataframe[col].nunique() > car_th]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car] 

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes  in ['int64', 'float64']] #bu sayısal değişkenleri verir
    num_cols = [col for col in num_cols if col not in cat_cols] 

    print(f'Observetions: {dataframe.shape[0]}')
    print(f'Variables: {dataframe.shape[1]}')
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car

In [49]:
#Değişkenleri tipleirne göre yakalıyoruz
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observetions: 891
Variables: 22
cat_cols: 13
num_cols: 5
cat_but_car: 3
num_but_cat: 7


In [51]:
num_cols

['PASSENGERID', 'AGE', 'FARE', 'NEW_NAME_COUNT', 'NEW_AGE_PCLASS']

In [53]:
num_cols = [col for col in num_cols if 'PASSENGERID' not in col ]# ıd bizim için ölçüm depğeri taşımadıgı için cıkardık

In [55]:
num_cols

['AGE', 'FARE', 'NEW_NAME_COUNT', 'NEW_AGE_PCLASS']

In [57]:
cat_but_car

['NAME', 'TICKET', 'CABIN']

<span style="color: blue; font-weight: bold;">2. ADIM:AYKIRI DEĞER </span>

In [63]:
def outlier_tresholds(dataframe, col_name, q1=0.25, q3=0.75):
    q1 = dataframe[col_name].quantile(q1)
    q3 = dataframe[col_name].quantile(q3)
    IQR = q3 -q1
    up_limit = q3 + 1.5*IQR
    low_limit = q1 - 1.5*IQR
    return low_limit, up_limit

In [65]:
def check_outliers(dataframe, col_name):
    low_limit, up_limit = outlier_tresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] < low_limit)|(dataframe[col_name] > up_limit)].any(axis=None):
        return True
    else:
        return False

In [71]:
#Aykırı değer var mı kontrol ettik
for col in num_cols:
    print(col,check_outliers(df, col))

AGE True
FARE True
NEW_NAME_COUNT True
NEW_AGE_PCLASS True


In [73]:
def replace_with_thresholds(dataframe, col_name):
    low_limit, up_limit = outlier_tresholds(dataframe, col_name)
    dataframe.loc[(dataframe[col_name] < low_limit), col_name] = low_limit
    dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit

In [75]:
#Aykırı değerkleri eşik değerlere baskılama
for col in num_cols:
    replace_with_thresholds(df, col)
  

In [77]:
#Oldu mu diye kontrol ettik.
for col in num_cols:
    print(col,check_outliers(df, col))

AGE False
FARE False
NEW_NAME_COUNT False
NEW_AGE_PCLASS False


<span style="color: blue; font-weight: bold;">3. ADIM:EKSİK DEĞER </span>

In [83]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns

In [85]:
#Hangi değişkenlerde eksik değer var
missing_values_table(df, na_name=True)

                n_miss  ratio
CABIN              687 77.100
AGE                177 19.870
NEW_AGE_PCLASS     177 19.870
NEW_AGE_CAT        177 19.870
NEW_SEX_CAT        177 19.870
EMBARKED             2  0.220


['AGE', 'CABIN', 'EMBARKED', 'NEW_AGE_PCLASS', 'NEW_AGE_CAT', 'NEW_SEX_CAT']

In [91]:
df.drop('CABIN', inplace=True, axis=1)#cabın değişkei üzerinden yeni değişken türettiğimiz için sildik.

In [97]:
#istemediğimiz başka değişkenler de var onları da silelim
remove = ['TICKET','NAME']
df.drop(remove, inplace=True, axis=1)

In [103]:
df['AGE'] = df['AGE'].fillna(df.groupby('NEW_TITLE').agg({'AGE':'median'}))

TypeError: "value" parameter must be a scalar, dict or Series, but you passed a "DataFrame"

* Yukarıdaki kullanımın doğru olmamasının ana nedeni, groupby().agg() ifadesinin DataFrame döndürmesi, ama .fillna() metodunun Series veya tek bir değer** beklemesidir.


In [106]:
df['AGE'] = df['AGE'].fillna(df.groupby('NEW_TITLE')['AGE'].transform('median'))

In [108]:
missing_values_table(df, na_name=True)

                n_miss  ratio
NEW_AGE_PCLASS     177 19.870
NEW_AGE_CAT        177 19.870
NEW_SEX_CAT        177 19.870
EMBARKED             2  0.220


['EMBARKED', 'NEW_AGE_PCLASS', 'NEW_AGE_CAT', 'NEW_SEX_CAT']

* Yaş değişkenin eksk değerlerini giderdik fakat yaş değişkeni üzerinden elde ettiğimiz değişkenlerde halen eksik değer var. Bunu gidermek için bu değişkenleri baştan oluşturmalıyız.

In [111]:
df["NEW_AGE_PCLASS"] = df["AGE"] * df["PCLASS"]

df.loc[(df['AGE'] < 18), 'NEW_AGE_CAT'] = 'young'
df.loc[(df['AGE'] >= 18) & (df['AGE'] < 56), 'NEW_AGE_CAT'] = 'mature'
df.loc[(df['AGE'] >= 56), 'NEW_AGE_CAT'] = 'senior'

df.loc[(df['SEX'] == 'male') & (df['AGE'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['SEX'] == 'male') & (df['AGE'] > 21) & (df['AGE'] < 50), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['SEX'] == 'male') & (df['AGE'] >= 50), 'NEW_SEX_CAT'] = 'seniormale'
df.loc[(df['SEX'] == 'female') & (df['AGE'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['SEX'] == 'female') & (df['AGE'] > 21) & (df['AGE'] < 50), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['SEX'] == 'female') & (df['AGE'] >= 50), 'NEW_SEX_CAT'] = 'seniorfemale'

In [115]:
missing_values_table(df, na_name=True)

          n_miss  ratio
EMBARKED       2  0.220


['EMBARKED']

In [117]:
#Eksik değer olan kategorik değişkenleri doldurduk.
df = df.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= 10) else x, axis=0)


<span style="color: blue; font-weight: bold;">4. ADIM:LABEL ENCODING </span>

In [131]:
df.head()

,PASSENGERID,SURVIVED,PCLASS,SEX,AGE,SIBSP,PARCH,FARE,EMBARKED,NEW_CABIN_BOOL,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT,NEW_NAME_DR,NEW_TITLE,NEW_FAMILY_SIZE,NEW_AGE_PCLASS,NEW_IS_ALONE,NEW_AGE_CAT,NEW_SEX_CAT
0,1,0,3,male,22.000,1,0,7.250,S,0,23,4,0,Mr,2,66.000,NO,mature,maturemale
1,2,1,1,female,38.000,1,0,65.634,C,1,45,7,0,Mrs,2,38.000,NO,mature,maturefemale
2,3,1,3,female,26.000,0,0,7.925,S,0,22,3,0,Miss,1,78.000,YES,mature,maturefemale
3,4,1,1,female,35.000,1,0,53.100,S,1,44,7,0,Mrs,2,35.000,NO,mature,maturefemale
4,5,0,3,male,35.000,0,0,8.050,S,0,24,4,0,Mr,1,105.000,YES,mature,maturemale


In [153]:
#ilk olara 2 sınıflı kategorik değişkenleri seçelim
binary_cols = [col for col in df.columns if df[col].dtype=='O' and df[col].nunique() ==2]

In [155]:
binary_cols

['SEX', 'NEW_IS_ALONE']

In [157]:
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[col] = labelencoder.fit_transform(dataframe[col])
    return dataframe

In [159]:
for col in binary_cols:
    df = label_encoder(df, col)

<span style="color: blue; font-weight: bold;">5. ADIM:RARE ENCODER </span>

In [163]:
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

In [165]:
rare_analyser(df, "SURVIVED", cat_cols)

SEX : 2
     COUNT  RATIO  TARGET_MEAN
SEX                           
0      314  0.352        0.742
1      577  0.648        0.189


EMBARKED : 3
          COUNT  RATIO  TARGET_MEAN
EMBARKED                           
C           168  0.189        0.554
Q            77  0.086        0.390
S           646  0.725        0.339


NEW_TITLE : 17
           COUNT  RATIO  TARGET_MEAN
NEW_TITLE                           
Capt           1  0.001        0.000
Col            2  0.002        0.500
Countess       1  0.001        1.000
Don            1  0.001        0.000
Dr             7  0.008        0.429
Jonkheer       1  0.001        0.000
Lady           1  0.001        1.000
Major          2  0.002        0.500
Master        40  0.045        0.575
Miss         182  0.204        0.698
Mlle           2  0.002        1.000
Mme            1  0.001        1.000
Mr           517  0.580        0.157
Mrs          125  0.140        0.792
Ms             1  0.001        1.000
Rev            6  0.007    

In [173]:
def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

In [175]:
df = rare_encoder(df, 0.01)

In [177]:
rare_analyser(df, "SURVIVED", cat_cols)

SEX : 2
     COUNT  RATIO  TARGET_MEAN
SEX                           
0      314  0.352        0.742
1      577  0.648        0.189


EMBARKED : 3
          COUNT  RATIO  TARGET_MEAN
EMBARKED                           
C           168  0.189        0.554
Q            77  0.086        0.390
S           646  0.725        0.339


NEW_TITLE : 5
           COUNT  RATIO  TARGET_MEAN
NEW_TITLE                           
Master        40  0.045        0.575
Miss         182  0.204        0.698
Mr           517  0.580        0.157
Mrs          125  0.140        0.792
Rare          27  0.030        0.444


NEW_IS_ALONE : 2
              COUNT  RATIO  TARGET_MEAN
NEW_IS_ALONE                           
0               354  0.397        0.506
1               537  0.603        0.304


NEW_AGE_CAT : 3
             COUNT  RATIO  TARGET_MEAN
NEW_AGE_CAT                           
mature         735  0.825        0.363
senior          39  0.044        0.308
young          117  0.131        0.538


NEW_

<span style="color: blue; font-weight: bold;">6. ADIM:ONE HOT ENCODING </span>

In [181]:
ohe_cols = [col for col in df.columns if 10>= df[col].nunique()>2]#İKİDEN FAZLA 10 DAN AZ OLAN DEĞİŞKENLERİ SECTİK

In [183]:
ohe_cols 

['PCLASS',
 'SIBSP',
 'PARCH',
 'EMBARKED',
 'NEW_NAME_WORD_COUNT',
 'NEW_TITLE',
 'NEW_FAMILY_SIZE',
 'NEW_AGE_CAT',
 'NEW_SEX_CAT']

In [185]:
def one_hot_encoder(dataframe, cat_col, drop_first=True):
    dataframe = pd.get_dummies(dataframe,columns = cat_col, drop_first = drop_first)
    return dataframe

In [191]:
df = one_hot_encoder(df, ohe_cols)

In [193]:
df.head()

,PASSENGERID,SURVIVED,SEX,AGE,FARE,NEW_CABIN_BOOL,NEW_NAME_COUNT,NEW_NAME_DR,NEW_AGE_PCLASS,NEW_IS_ALONE,PCLASS_2,PCLASS_3,SIBSP_1,SIBSP_2,SIBSP_3,SIBSP_4,SIBSP_5,SIBSP_8,PARCH_1,PARCH_2,PARCH_3,PARCH_4,PARCH_5,PARCH_6,EMBARKED_Q,EMBARKED_S,NEW_NAME_WORD_COUNT_4,NEW_NAME_WORD_COUNT_5,NEW_NAME_WORD_COUNT_6,NEW_NAME_WORD_COUNT_7,NEW_NAME_WORD_COUNT_8,NEW_NAME_WORD_COUNT_9,NEW_NAME_WORD_COUNT_14,NEW_TITLE_Miss,NEW_TITLE_Mr,NEW_TITLE_Mrs,NEW_TITLE_Rare,NEW_FAMILY_SIZE_2,NEW_FAMILY_SIZE_3,NEW_FAMILY_SIZE_4,NEW_FAMILY_SIZE_5,NEW_FAMILY_SIZE_6,NEW_FAMILY_SIZE_7,NEW_FAMILY_SIZE_8,NEW_FAMILY_SIZE_11,NEW_AGE_CAT_senior,NEW_AGE_CAT_young,NEW_SEX_CAT_maturemale,NEW_SEX_CAT_seniorfemale,NEW_SEX_CAT_seniormale,NEW_SEX_CAT_youngfemale,NEW_SEX_CAT_youngmale
0,1,0,1,22.000,7.250,0,23,0,66.000,0,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,2,1,0,38.000,65.634,1,45,0,38.000,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,3,1,0,26.000,7.925,0,22,0,78.000,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,4,1,0,35.000,53.100,1,44,0,35.000,0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,5,0,1,35.000,8.050,0,24,0,105.000,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False


In [195]:
df.shape

(891, 52)

In [197]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observetions: 891
Variables: 52
cat_cols: 44
num_cols: 5
cat_but_car: 0
num_but_cat: 2


In [199]:
#oluşturduğumuz değişkenler gercekten işe gerekli mi değil mi analiz etmeye çalışıyoruz.
rare_analyser(df, "SURVIVED", cat_cols)

PCLASS_2 : 2
          COUNT  RATIO  TARGET_MEAN
PCLASS_2                           
False       707  0.793        0.361
True        184  0.207        0.473


PCLASS_3 : 2
          COUNT  RATIO  TARGET_MEAN
PCLASS_3                           
False       400  0.449        0.557
True        491  0.551        0.242


SIBSP_1 : 2
         COUNT  RATIO  TARGET_MEAN
SIBSP_1                           
False      682  0.765        0.337
True       209  0.235        0.536


SIBSP_2 : 2
         COUNT  RATIO  TARGET_MEAN
SIBSP_2                           
False      863  0.969        0.381
True        28  0.031        0.464


SIBSP_3 : 2
         COUNT  RATIO  TARGET_MEAN
SIBSP_3                           
False      875  0.982        0.386
True        16  0.018        0.250


SIBSP_4 : 2
         COUNT  RATIO  TARGET_MEAN
SIBSP_4                           
False      873  0.980        0.388
True        18  0.020        0.167


SIBSP_5 : 2
         COUNT  RATIO  TARGET_MEAN
SIBSP_5            

In [201]:
#kullanışsız sutunlr yani oran olarak çok düşük olan sınıflar
useless_cols = [col for col in df.columns if df[col].nunique() == 2 and
                (df[col].value_counts() / len(df) < 0.01).any(axis=None)]


In [203]:
useless_cols # bu oranları düşük olan değişkenleri silip silmemek bizim tercihimize kalan bir duurm

['SIBSP_5',
 'SIBSP_8',
 'PARCH_3',
 'PARCH_4',
 'PARCH_5',
 'PARCH_6',
 'NEW_NAME_WORD_COUNT_9',
 'NEW_NAME_WORD_COUNT_14',
 'NEW_FAMILY_SIZE_8',
 'NEW_FAMILY_SIZE_11']

<span style="color: blue; font-weight: bold;">7. STANDARTLAŞTIRMA </span>

* Bu veri setinde gerekli bir adım değildir. Gerekli olması durumunda aşağıdaki gibi yapılmalıdır.

In [208]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

df[num_cols].head()


(891, 52)

<span style="color: blue; font-weight: bold;">8. MODEL KURMA </span>

* Bu kısım makine öğrenmesinde detaylı ele alınacağı için yapılmamıştır.